### Workflow for getting composition for each roi annotation for each DLmask (DLmask generated from entire mask of ndpi):

In [5]:
OPENSLIDE_PATH = r'C:\Users\Kevin\Desktop\openslide-win64-20221217\bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [6]:
# import pandas as pd
import numpy as np
import os
from PIL import Image
import os
import cv2
Image.MAX_IMAGE_PIXELS = 1000000000000000000
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import openslide
from PIL import Image
from matplotlib import pyplot as plt
import skimage.measure
import matplotlib.pyplot as plt
import scipy.stats as stats
import torchvision.transforms as transforms
from glob import glob
from time import time
from skimage.measure import label
import math

In [7]:
# # quick pandas work to see if ndpi's in image list have the DL segmented masks!
# read xml files for roi annotations
roi_src = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi'
# read DL mask files for DL segmented masks
dl_src = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
# read image list and fill in DL_segmented column (0=DLmask missing, 1=DLmask exists)
# im_list = r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207.xlsx'

In [22]:
roi_list = [x for x in os.listdir(roi_src) if x.endswith('.xml')] #301 rows
dl_list = [x for x in os.listdir(dl_src) if x.endswith('.tif')] #301 rows
roi_complete_path = [os.path.join(roi_src,x) for x in roi_list]
dl_complete_path = [os.path.join(dl_src,x) for x in dl_list]
roi_raw = [x.replace(".xml","") for x in roi_list]
dl_raw = [x.replace(".tif","") for x in dl_list]

In [23]:
# # image_list_csv_raw = pd.read_excel(r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207.xlsx')

In [24]:
# # dl_list = [x.replace(".tif","") for x in dl_list]

In [25]:
# # def check_DLexist(row):
#     row = row.replace('.ndpi','')
#     return row in dl_list

In [26]:
# # image_list_csv_raw['DL_segmented'] = image_list_csv_raw.apply(lambda row : check_DLexist(row['filename']), axis = 1)

In [27]:
# # image_list_csv_raw.head()

In [28]:
# # image_list_csv_raw.to_excel(r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207_v2.xlsx')

In [29]:
# # First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
def xml_to_df(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [ ]:
roi= readim(roipath)
szdl = size(dlmask)
roi2 = resize(roi,szdl)

In [30]:
# # Then input original image and the coord_df to output the mask with unique annotations (1..N, N = 12 in this case):
def create_mask_multi_annot(xml_path, image_path):
    slide = openslide.open_slide(image_path)
    target_dim = [math.ceil(x / 2.26459) for x in slide.dimensions]
    # target_level = slide.get_best_level_for_downsample(downsample_factor)
    # target_dim = slide.level_dimensions[target_level]
    # rsf = [x/y for x,y in zip(slide.dimensions,target_dim)] #resize factor
    mask = np.zeros((target_dim), dtype = np.uint8)
    coord_df = xml_to_df(xml_path) #use function above
    for idx, row in coord_df.iterrows():
        yy = row.X.astype(float).astype('int32')
        yy = [math.ceil(n / 2.26459) for n in yy]
        xx = row.Y.astype(float).astype('int32')
        xx = [math.ceil(m / 2.26459) for m in xx]
        contours = np.array(list(zip(xx,yy)))
        mask = cv2.fillPoly(mask, pts=[contours.astype(int)], color=(int(coord_df.index.values[idx]) + 1))
    mask = mask.T
    return mask

In [31]:
# find intersection of ndpi, roi, and dl -> A ∩ B ∩ C = (A ∩ B) ∩ (B ∩ C), roi = A, dl = B, ndpi = C

ndpi_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi'
ndpi_list = [x for x in os.listdir(ndpi_src) if x.endswith('.ndpi')] #301 rows
ndpi_raw = [x.replace(".ndpi","") for x in ndpi_list]

roi_dl_overlap = set(roi_raw).intersection(set(dl_raw)) #216
dl_ndpi_overlap = set(dl_raw).intersection(set(ndpi_raw)) #222
all_overlap = set(roi_dl_overlap).intersection(set(dl_ndpi_overlap)) #216

dl_complete_path_new = [os.path.join(dl_src,x) for x in all_overlap]
roi_complete_path_new = [os.path.join(roi_src,x) for x in all_overlap]
ndpi_complete_path_new = [os.path.join(ndpi_src,x) for x in all_overlap]
dl_complete_path_new = [x + ".tif" for x in dl_complete_path_new]
roi_complete_path_new = [x + ".xml" for x in roi_complete_path_new]
ndpi_complete_path_new = [x + ".ndpi" for x in ndpi_complete_path_new]
save_name = [os.path.basename(x) for x in ndpi_complete_path_new]

In [32]:
len(save_name)

216

In [33]:
# # read a first DLmask and its corresponding roi xml file
roi_png_dst = os.path.join(roi_src,'roi_png_1x') # destination to save grayscale labeled mask
class_list = list(['corneum','spinosum','hairshaft','hairfollicle','smoothmuscle','oil','sweat','nerve','bloodvessel','ecm','fat','white'])
comps_df = pd.DataFrame(index = class_list)
for idx in range(0,len(dl_complete_path_new)):
    print(dl_complete_path_new[idx])
    DLarr = Image.open(dl_complete_path_new[idx])
    DLarr = np.array(DLarr)
    mask = create_mask_multi_annot(roi_complete_path_new[idx], ndpi_complete_path_new[idx]) #, downsample_factor= 2.264596)
    roi_png = Image.fromarray(mask)
    roi_png.save(os.path.join(roi_png_dst,save_name[idx].replace("ndpi","png"))) # save labeled mask
#for example of roi_png: "\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_20rsf", roi_png is an image with grayscale labeled mask, save this.
    for sectionID in range(1,np.max(roi_png)+1):
        roi_png = np.array(roi_png)
        roi_shape = roi_png.shape
        DLarr = np.array(DLarr)
        DLarr_shape = DLarr.shape
        if roi_shape != DLarr_shape:
            #resize DLarr to roi_shape
            roi_png_changed = cv2.resize(roi_png,(DLarr_shape[1],DLarr_shape[0]),interpolation = cv2.INTER_LINEAR)
            sectionID_ra = np.array([sectionID])
            bool = np.isin(roi_png_changed, sectionID_ra)
            DL_roi = np.multiply(DLarr, bool)
            Image.fromarray(DL_roi).save(os.path.join(r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi',str(save_name[idx].replace(".ndpi",""))) + "_" + str(sectionID) + ".png") #save labeled mask per roi (dl_crop_roi)
            DL_roi = DL_roi[DL_roi > 0] #background 0 shouldn't count in the composition
            DL_roi_flatten = DL_roi.flatten()
            hist = np.bincount(DL_roi_flatten) #dl mask
            hist = hist/np.sum(hist)*100
            count_list = list(hist)
            del(count_list[0])
            count_list = [np.round(x,3) for x in count_list]

            ratio_class_dict = dict(zip(class_list,count_list))
            tmp_df = pd.DataFrame.from_dict(ratio_class_dict,orient='index')
            comps_df = pd.concat([comps_df,pd.DataFrame.from_dict(tmp_df)],axis=1)

        else:
            sectionID_ra = np.array([sectionID])
            bool = np.isin(roi_png, sectionID_ra)
            DL_roi = np.multiply(DLarr, bool)
            Image.fromarray(DL_roi).save(os.path.join(r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi',str(save_name[idx].replace(".ndpi",""))) + "_" + str(sectionID) + ".png") #save labeled mask per roi (dl_crop_roi)
            DL_roi = DL_roi[DL_roi > 0] #background 0 shouldn't count in the composition
            DL_roi_flatten = DL_roi.flatten()
            hist = np.bincount(DL_roi_flatten) #dl mask
            hist = hist/np.sum(hist)*100
            count_list = list(hist)
            del(count_list[0])
            count_list = [np.round(x,3) for x in count_list]

            ratio_class_dict = dict(zip(class_list,count_list))
            tmp_df = pd.DataFrame.from_dict(ratio_class_dict,orient='index')
            comps_df = pd.concat([comps_df,pd.DataFrame.from_dict(tmp_df)],axis=1)

\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um\2022-07-07 17.19.31.tif



KeyboardInterrupt



In [ ]:
comps_df
comps_df_T = comps_df.T
import glob
order_src = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeled_mask_per_roi'
order_files = list(filter(os.path.isfile, glob.glob(order_src + "\*")))
order_files.sort(key=os.path.getctime)

index_list = [x[-6:] for x in order_files]
index_list = [x[1] for x in index_list]
index_list = [int(x) for x in index_list]
full_path_list = [x[:-6] for x in order_files]
full_path_list = [x + ".png" for x in full_path_list]

comps_df_T["filepath"] = full_path_list
comps_df_T["index"] = index_list
comps_df_T.to_excel(r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_roi_composition.xlsx')